In [10]:
import search
reload(search)

# reopen index, but reload it instead of creating it afresh
udacity_se = search.UdacitySearchEngine(create=False)
    
# dictionary of results
results = udacity_se.search("apple")
print results

[{'slug': u'grand-central-dispatch-gcd--ud576', 'title': u'Grand Central Dispatch (GCD)'}, {'slug': u'intro-to-ios-app-development-with-swift--ud585', 'title': u'Intro to iOS App Development with Swift'}, {'slug': u'swift-for-beginners--ud1022', 'title': u'Swift for Beginners'}, {'slug': u'server-side-swift--ud1031', 'title': u'Server-Side Swift'}, {'slug': u'learn-swift-programming-syntax--ud902', 'title': u'Learn Swift Programming Syntax'}, {'slug': u'ios-persistence-and-core-data--ud325', 'title': u'iOS Persistence and Core Data'}]


In [21]:
import search
reload(search)

# reopen index, but reload it instead of creating it afresh
se = search.HarvardXSearchEngine(create=False)
    
# dictionary of results
results = se.search("Psychology")
print results

[{'course_id': u'course-v1:HarvardX+GSE4x+2T2016', 'display_name': u'Kathleen Hoover-Dempsey Bio'}, {'course_id': u'course-v1:HarvardX+GSE4x+2T2016', 'display_name': u'Additional Resources'}, {'course_id': u'course-v1:HarvardX+GSE4x+2T2016', 'display_name': u'Watch: Parental Motivation for Engagement'}, {'course_id': u'course-v1:HarvardX+GSE4x+2T2016', 'display_name': u'Additional Resources'}, {'course_id': u'course-v1:HarvardX+PH201x+2013_SOND', 'display_name': u'Meet the Teaching Assistants: Felicia Browne'}, {'course_id': u'course-v1:HarvardX+GSE2x+1T2016', 'display_name': u'Additional Resources'}, {'course_id': u'course-v1:HarvardX+GSE2x+1T2016', 'display_name': u'Intro to Neuroeducation'}, {'course_id': u'course-v1:HarvardX+1368.2x+2T2016', 'display_name': u'Glossary'}, {'course_id': u'course-v1:HarvardX+GSE4x+2T2016', 'display_name': u'Explore the Research'}, {'course_id': u'course-v1:HarvardX+AmPoX.6+2T2016', 'display_name': u''}]


In [36]:
# compare query rewriting to the normal results 
import rewriter
reload(rewriter)

import search
reload(search)

searches = [
    "neural networks",
    "swift",
    "venture capital",
    "object-oriented programming",
    "deep learning",
    "macOS"
]


udacity_se = search.UdacitySearchEngine(create=False)
rw = rewriter.WikipediaRewriter()
cw = rewriter.ControlRewriter()

wikidacity = rewriter.RewritingSearchEngine(rw, udacity_se)
plaindacity = rewriter.RewritingSearchEngine(cw, udacity_se)

for query in searches:
    # normal results
    num_plain_results = len(plaindacity.search(query))
    
    # with wikipedia rewriting
    num_wiki_results = len(wikidacity.search(query))
    
    print (query, num_plain_results, num_wiki_results)

('neural networks', 2, 2)
('swift', 9, 9)
('venture capital', 1, 1)
('object-oriented programming', 6, 6)
('deep learning', 1, 1)
('macOS', 0, 0)


<module 'rewriter' from 'rewriter.py'>

In [64]:
import rewriter
reload(rewriter)

wv = rewriter.Word2VecRewriter(create=False)

In [65]:
worddacity = rewriter.RewritingSearchEngine(wv, udacity_se)

In [67]:
for query in searches:
    # normal results
    num_plain_results = len(plaindacity.search(query))

    num_word_results = len(worddacity.search(query))
    
    print (query, num_plain_results, num_word_results)

('neural networks', 2, 2)
('swift', 9, 11)
('venture capital', 1, 1)
('object-oriented programming', 6, 6)
('deep learning', 1, 1)
('macOS', 0, 0)


In [69]:
# compare query rewriting to the normal results 
import rewriter
reload(rewriter)

import search
reload(search)

hx_se = search.HarvardXSearchEngine(create=False)

cw = rewriter.ControlRewriter()
rw = rewriter.WikipediaRewriter()
wv = rewriter.Word2VecRewriter(create=False)

rewrite_engines = [
    rewriter.RewritingSearchEngine(cw, hx_se),
    rewriter.RewritingSearchEngine(rw, hx_se),
    rewriter.RewritingSearchEngine(wv, hx_se)
]

In [76]:
searches = [
    "confirmation bias",
    "biochemistry",
    "deep learning"
]



for query in searches:
    result_nums = [len(qrse.search(query)) for qrse in rewrite_engines]
    
    print (query, result_nums)

('confirmation bias', [2, 49, 2])
('biochemistry', [10, 37, 101])
('deep learning', [10, 13, 10])
